# Part 1.2: Calculus for Deep Learning — The Formula 1 Edition

Calculus is essential for understanding how neural networks learn. The key insight: **learning = optimization**, and optimization requires derivatives.

But here's the thing: **F1 engineers live and breathe calculus every race weekend**. When an engineer asks "how much faster will we go if we reduce the rear wing angle by one degree?" — that's a derivative. When a strategist asks "which setup parameter should we change to improve lap time the most?" — that's a gradient. And when the team iteratively tweaks the car setup between practice sessions to find the optimal configuration — that's gradient descent.

Every tenth of a second on track is found through the same mathematical machinery that trains neural networks.

## Learning Objectives
- [ ] Compute partial derivatives of multivariate functions
- [ ] Apply the chain rule to composite functions
- [ ] Understand gradients as directions of steepest ascent
- [ ] Implement gradient descent from scratch

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

## 1. Derivatives: The Core Concept

The **derivative** measures the rate of change of a function:

$$f'(x) = \frac{df}{dx} = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}$$

**Intuition**: The derivative tells you how much the output changes when you slightly change the input.

**F1 analogy:** The derivative is the speedometer of calculus. Your car's speed is the derivative of its position along the track — it tells you how fast your position is changing at any given instant. Acceleration is the derivative of speed. Tire degradation rate is the derivative of tire performance over laps. Every "rate of change" question on the pit wall is a derivative question.

### Why This Matters for Deep Learning

If `loss = f(weights)`, then the derivative tells us:
- **Which direction** to change weights to reduce loss
- **How much** each weight affects the loss

### Deep Dive: What Does a Derivative Really Mean?

The derivative answers a fundamental question: **"If I wiggle the input a tiny bit, how much does the output wiggle?"**

Think of it like this:
- You're turning a dial (input x)
- A meter responds (output f(x))
- The derivative tells you: "For each unit I turn the dial, how many units does the meter move?"

**F1 analogy:** Imagine you're the race engineer adjusting the front wing angle on your driver's car. You tweak it by one degree (the "wiggle"). The derivative tells you: "For that one degree of wing change, how many km/h did your top speed change?" or "How many tenths did your cornering speed improve?" The derivative is the sensitivity of performance to your adjustment.

#### The Formal Definition Unpacked

$$f'(x) = \lim_{h \to 0} \frac{f(x + h) - f(x)}{h}$$

| Component | Meaning | F1 Analogy |
|-----------|---------|------------|
| $f(x + h)$ | Output after nudging input by tiny amount h | Lap time after a small setup change |
| $f(x)$ | Original output | Lap time with current setup |
| $f(x + h) - f(x)$ | Change in output (how much the meter moved) | Time gained or lost from the change |
| $h$ | Change in input (how much we turned the dial) | Size of the setup adjustment |
| $\frac{f(x+h) - f(x)}{h}$ | **Rate of change** = output change per unit input change | Seconds gained per degree of wing angle |
| $\lim_{h \to 0}$ | Take h infinitesimally small (instantaneous rate) | The exact sensitivity at this precise setting |

#### Why Do We Care About Rate of Change?

| Context | What the Derivative Tells Us | F1 Parallel |
|---------|------------------------------|-------------|
| **Physics** | Velocity = derivative of position. "How fast am I moving right now?" | Instantaneous speed through a corner |
| **Economics** | Marginal cost = derivative of total cost. "Cost of making one more unit?" | Cost per tenth of a second of downforce |
| **Machine Learning** | Gradient of loss = derivative of loss w.r.t. weights. "How does changing this weight affect the error?" | "How does changing wing angle affect lap time?" |

In ML specifically: **Derivatives tell us which direction to adjust weights to reduce error.**

In F1 specifically: **Derivatives tell the engineer which direction to adjust the setup to reduce lap time.**

In [ ]:
# Numerical derivative approximation
def numerical_derivative(f, x, h=1e-5):
    """Approximate derivative using finite differences."""
    return (f(x + h) - f(x - h)) / (2 * h)

# Example: f(x) = x^2 — Think of x as wing angle, f(x) as drag force
wing_angle = lambda x: x**2
wing_angle_derivative = lambda x: 2*x  # We know this analytically

angle = 3.0
print(f"f(x) = x² at x = {angle}")
print(f"  (Think: drag as a function of wing angle)")
print(f"Numerical derivative: {numerical_derivative(wing_angle, angle):.6f}")
print(f"Analytical derivative: {wing_angle_derivative(angle):.6f}")

In [ ]:
# Interactive visualization: Tangent lines at multiple points
# Shows how the derivative (slope) changes across the function
# F1 context: Think of this as how the "sensitivity" of lap time to a setup 
# parameter changes depending on where you are in the setup range

def plot_multiple_tangents(f, f_prime, x_range, points, title):
    """
    Plot a function with tangent lines at multiple points.
    This visualizes how the derivative changes across the function.
    """
    x = np.linspace(x_range[0], x_range[1], 200)
    y = f(x)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left plot: Function with tangent lines
    axes[0].plot(x, y, 'b-', linewidth=2, label='f(x)')
    
    colors = plt.cm.Reds(np.linspace(0.3, 0.9, len(points)))
    
    for x0, color in zip(points, colors):
        y0 = f(x0)
        slope = f_prime(x0)
        
        # Tangent line: y = f(x0) + f'(x0)(x - x0)
        x_tangent = np.linspace(x0 - 1.5, x0 + 1.5, 50)
        y_tangent = y0 + slope * (x_tangent - x0)
        
        axes[0].plot(x_tangent, y_tangent, '--', color=color, linewidth=1.5, alpha=0.8)
        axes[0].scatter([x0], [y0], color=color, s=80, zorder=5)
        axes[0].annotate(f'slope={slope:.2f}', xy=(x0, y0), xytext=(x0+0.3, y0+0.5),
                        fontsize=9, color=color)
    
    axes[0].set_xlabel('Setup Parameter')
    axes[0].set_ylabel('Performance')
    axes[0].set_title(f'{title}\nTangent lines show instantaneous rate of change')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([min(y) - 1, max(y) + 2])
    
    # Right plot: The derivative function itself
    y_prime = f_prime(x)
    axes[1].plot(x, y_prime, 'r-', linewidth=2, label="f'(x) (derivative)")
    axes[1].axhline(y=0, color='k', linewidth=0.5)
    
    for x0, color in zip(points, colors):
        slope = f_prime(x0)
        axes[1].scatter([x0], [slope], color=color, s=80, zorder=5)
    
    axes[1].set_xlabel('Setup Parameter')
    axes[1].set_ylabel("f'(x) — Sensitivity")
    axes[1].set_title("The Derivative Function\nShows the sensitivity at every point")
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Example 1: f(x) = x^2 (parabola) — like drag vs wing angle
f = lambda x: x**2
f_prime = lambda x: 2*x
plot_multiple_tangents(f, f_prime, x_range=(-3, 3), 
                       points=[-2, -1, 0, 1, 2], 
                       title='f(x) = x² — Drag vs Wing Angle')

print("Key observations for f(x) = x²:")
print("- At x=0: slope is 0 (bottom of the parabola - minimum!)")
print("  F1: At the optimal wing angle, tiny changes don't affect drag")
print("- Negative x: slope is negative (function decreasing)")
print("- Positive x: slope is positive (function increasing)")
print("- Slope magnitude increases as we move away from 0")
print("  F1: The further from optimal, the more sensitive the car becomes")

In [ ]:
# Visualize derivative as slope of tangent line
def plot_tangent(f, f_prime, x0, title):
    """Plot function with tangent line at x0."""
    x = np.linspace(x0 - 2, x0 + 2, 100)
    y = f(x)
    
    # Tangent line: y = f(x0) + f'(x0)(x - x0)
    slope = f_prime(x0)
    tangent = f(x0) + slope * (x - x0)
    
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, 'b-', linewidth=2, label='f(x)')
    plt.plot(x, tangent, 'r--', linewidth=2, label=f'Tangent (slope = {slope:.2f})')
    plt.scatter([x0], [f(x0)], color='red', s=100, zorder=5)
    plt.xlabel('Wing Angle (degrees)')
    plt.ylabel('Drag Coefficient')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# f(x) = x² at different points
f = lambda x: x**2
f_prime = lambda x: 2*x

plot_tangent(f, f_prime, 1.5, "Drag vs Wing Angle: f(x) = x² with tangent at x = 1.5\nSlope tells us how sensitive drag is to wing changes at this point")

In [ ]:
# Example 2: A more complex function - sine wave
# F1 context: Oscillating performance, like tire grip over a lap 
# (grip cycles as tire temp rises and falls through corners)
f = lambda x: np.sin(x)
f_prime = lambda x: np.cos(x)
plot_multiple_tangents(f, f_prime, x_range=(-2*np.pi, 2*np.pi), 
                       points=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi], 
                       title='f(x) = sin(x) — Oscillating Tire Grip')

print("\nKey observations for f(x) = sin(x):")
print("- At peaks/troughs (x = +/-pi/2): slope is 0 (maxima/minima)")
print("  F1: When grip peaks or bottoms out, it's momentarily stable")
print("- At zero crossings (x = 0, +/-pi): slope is +/-1 (steepest)")
print("  F1: Grip is changing fastest during transitions")
print("- The derivative of sin(x) is cos(x) - shifted by pi/2!")

### Common Derivatives

| Function | Derivative | F1 Analogy |
|----------|------------|------------|
| $x^n$ | $nx^{n-1}$ | Power-law relationships (aero drag scales with speed squared) |
| $e^x$ | $e^x$ | Exponential tire degradation — rate of wear proportional to current wear |
| $\ln(x)$ | $1/x$ | Diminishing returns — each extra unit of downforce costs more drag |
| $\sin(x)$ | $\cos(x)$ | Oscillating grip through corner sequences |
| $\cos(x)$ | $-\sin(x)$ | Phase-shifted oscillations in suspension response |

### Activation Functions and Their Derivatives

These are critical for backpropagation!

**F1 analogy:** Activation functions are like driver response curves. A sigmoid is like brake feel — gentle initial response, strong in the middle, saturating at full lock. ReLU is like a rev limiter — nothing below threshold, linear above it.

In [ ]:
# Sigmoid and its derivative — like brake feel: gentle at first, strong in mid-range, saturates
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)  # Nice property!

# ReLU and its derivative — like a rev limiter: nothing below threshold, linear above
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

# Tanh and its derivative — like steering response: symmetric, saturates at extremes
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

# Plot them all
x = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

activations = [
    (sigmoid, sigmoid_derivative, 'Sigmoid\n(Brake Feel)'),
    (relu, relu_derivative, 'ReLU\n(Rev Limiter)'),
    (tanh, tanh_derivative, 'Tanh\n(Steering Response)')
]

for i, (func, deriv, name) in enumerate(activations):
    # Function
    axes[0, i].plot(x, func(x), 'b-', linewidth=2)
    axes[0, i].set_title(f'{name}')
    axes[0, i].set_xlabel('Input Signal')
    axes[0, i].axhline(y=0, color='k', linewidth=0.5)
    axes[0, i].axvline(x=0, color='k', linewidth=0.5)
    axes[0, i].grid(True, alpha=0.3)
    
    # Derivative
    axes[1, i].plot(x, deriv(x), 'r-', linewidth=2)
    axes[1, i].set_title(f'{name.split(chr(10))[0]} Derivative (Sensitivity)')
    axes[1, i].set_xlabel('Input Signal')
    axes[1, i].axhline(y=0, color='k', linewidth=0.5)
    axes[1, i].axvline(x=0, color='k', linewidth=0.5)
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key observations:")
print("- Sigmoid derivative max is 0.25 (causes vanishing gradients)")
print("  F1: Like over-assisted brakes — can't feel the road anymore")
print("- ReLU derivative is 0 or 1 (no vanishing gradient for positive x)")
print("  F1: Clean on/off response — either engaged or not")
print("- Tanh derivative max is 1 (better than sigmoid)")
print("  F1: Better steering feel, but still saturates at extremes")

---

## 2. Partial Derivatives

For functions of multiple variables, **partial derivatives** measure the rate of change with respect to one variable while holding others constant.

$$\frac{\partial f}{\partial x} = \lim_{h \to 0} \frac{f(x + h, y) - f(x, y)}{h}$$

**F1 analogy:** An F1 car has dozens of setup parameters — wing angles, ride height, spring stiffness, tire pressures, brake bias, differential settings. A partial derivative answers the question: **"If I change ONLY the front wing angle (holding everything else fixed), how much does the lap time change?"** This is exactly what engineers do during practice sessions — isolate one variable at a time to understand its individual effect.

### Example

For $f(x, y) = x^2 + 3xy + y^2$:

$$\frac{\partial f}{\partial x} = 2x + 3y$$
$$\frac{\partial f}{\partial y} = 3x + 2y$$

### Deep Dive: Why Does the Gradient Point "Uphill"?

This is one of the most important insights in optimization. Let's build intuition for WHY the gradient points in the direction of steepest increase.

#### The Gradient as a "Which Way is Up?" Detector

Imagine you're standing on a hilly surface (the function) and want to find the steepest uphill direction. The gradient is like a compass that always points uphill.

**F1 analogy:** Think of the car setup landscape as a terrain map, where altitude represents lap time. The gradient at your current setup is like your race engineer's gut feeling about which combination of changes would make the car slower the fastest. To make the car **faster**, you go in the **opposite** direction — down the gradient, toward lower lap times. This is exactly what gradient descent does.

**Mathematical Intuition:**

The gradient $\nabla f$ at a point gives you the direction where the function increases **most rapidly**.

Think about it:
- $\frac{\partial f}{\partial x}$ tells you: "If I move in the x-direction, how fast does f increase?"
- $\frac{\partial f}{\partial y}$ tells you: "If I move in the y-direction, how fast does f increase?"
- The gradient combines these: "The optimal uphill direction is a blend of these, weighted by how steep each direction is"

**F1 analogy:** If changing wing angle (x) improves lap time by 0.3s per degree, and changing ride height (y) improves it by 0.5s per mm, the gradient tells you: "The fastest way to improve is a blend of both, weighted by their individual sensitivities."

#### Directional Derivatives: Movement in Any Direction

If you move in direction $\mathbf{u}$ (a unit vector), the rate of change is:

$$\frac{\partial f}{\partial \mathbf{u}} = \nabla f \cdot \mathbf{u} = |\nabla f| \cos(\theta)$$

Where $\theta$ is the angle between gradient and movement direction.

| Direction relative to gradient | $\cos(\theta)$ | Rate of change | F1 Interpretation |
|-------------------------------|----------------|----------------|-------------------|
| Same as gradient ($\theta = 0$) | 1 | Maximum increase | Worst possible setup change (max lap time increase) |
| Perpendicular ($\theta = 90$) | 0 | No change (contour line) | A trade-off change — different but equally fast |
| Opposite ($\theta = 180$) | -1 | Maximum decrease | Best possible setup change (max lap time decrease) |

**This is why gradient descent works!** Moving opposite to the gradient gives maximum decrease.

In [ ]:
# Numerical partial derivatives
def partial_derivative(f, point, var_index, h=1e-5):
    """
    Compute partial derivative of f at point with respect to variable var_index.
    
    F1 context: This is like measuring the sensitivity of lap time to ONE 
    setup parameter while holding all others fixed.
    
    Args:
        f: Function taking array of variables (setup parameters)
        point: Array of variable values (current setup)
        var_index: Which variable to differentiate (which parameter to tweak)
        h: Step size (how big a tweak)
    """
    point = np.array(point, dtype=float)
    point_plus = point.copy()
    point_minus = point.copy()
    point_plus[var_index] += h
    point_minus[var_index] -= h
    return (f(point_plus) - f(point_minus)) / (2 * h)

# f(wing_angle, ride_height) = wing_angle² + 3*wing_angle*ride_height + ride_height²
# Think: lap time as a function of two setup parameters
def lap_time_model(p):
    wing_angle, ride_height = p
    return wing_angle**2 + 3*wing_angle*ride_height + ride_height**2

# Analytical partial derivatives
def dlap_d_wing(wing_angle, ride_height):
    return 2*wing_angle + 3*ride_height

def dlap_d_height(wing_angle, ride_height):
    return 3*wing_angle + 2*ride_height

# Test at point (2, 3) — wing angle=2, ride height=3
setup_point = [2, 3]
print(f"At setup point (wing_angle={setup_point[0]}, ride_height={setup_point[1]}):")
print(f"  d(lap_time)/d(wing_angle) numerical:  {partial_derivative(lap_time_model, setup_point, 0):.6f}")
print(f"  d(lap_time)/d(wing_angle) analytical: {dlap_d_wing(*setup_point):.6f}")
print(f"  d(lap_time)/d(ride_height) numerical:  {partial_derivative(lap_time_model, setup_point, 1):.6f}")
print(f"  d(lap_time)/d(ride_height) analytical: {dlap_d_height(*setup_point):.6f}")
print(f"\nInterpretation: ride_height has a bigger derivative ({dlap_d_height(*setup_point)}) than wing_angle ({dlap_d_wing(*setup_point)})")
print("=> Changing ride height would affect lap time more at this setup point")

In [ ]:
# Enhanced gradient field visualization with interactive exploration
# Shows gradient as arrows pointing uphill, with different movement directions
# F1: Gradient field over the setup landscape — arrows show "which way makes the car slower"

def visualize_gradient_directions():
    """
    Interactive visualization showing:
    1. Gradient field (arrows pointing uphill / toward slower lap times)
    2. How rate of change varies with direction
    3. Why opposite-to-gradient is the best descent direction
    """
    
    fig = plt.figure(figsize=(16, 5))
    
    # Define function: f(x,y) = x² + 0.5*y² (elliptical paraboloid)
    # F1: Lap time surface — wing_angle² + 0.5*ride_height²
    def f(x, y):
        return x**2 + 0.5*y**2
    
    def grad_f(x, y):
        return np.array([2*x, y])
    
    # Create grid
    x = np.linspace(-3, 3, 100)
    y = np.linspace(-3, 3, 100)
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    
    # Plot 1: 3D surface
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.7)
    ax1.set_xlabel('Wing Angle')
    ax1.set_ylabel('Ride Height')
    ax1.set_zlabel('Lap Time')
    ax1.set_title('Lap Time Surface\n(Bowl = optimal setup at center)')
    
    # Plot 2: Contour with gradient field
    ax2 = fig.add_subplot(132)
    contour = ax2.contour(X, Y, Z, levels=15, cmap=cm.viridis)
    ax2.clabel(contour, inline=True, fontsize=8)
    
    # Sparse grid for gradient arrows
    x_sparse = np.linspace(-2.5, 2.5, 8)
    y_sparse = np.linspace(-2.5, 2.5, 8)
    X_s, Y_s = np.meshgrid(x_sparse, y_sparse)
    
    U = 2 * X_s  # df/d(wing_angle)
    V = Y_s      # df/d(ride_height)
    
    # Normalize for visualization
    mag = np.sqrt(U**2 + V**2) + 1e-10
    U_norm = U / mag * 0.4
    V_norm = V / mag * 0.4
    
    ax2.quiver(X_s, Y_s, U_norm, V_norm, mag, cmap=cm.Reds, alpha=0.8)
    ax2.set_xlabel('Wing Angle')
    ax2.set_ylabel('Ride Height')
    ax2.set_title('Gradient Field Over Setup Space\nArrows point toward SLOWER lap times')
    ax2.set_aspect('equal')
    ax2.plot([0], [0], 'k*', markersize=15, label='Optimal Setup')
    ax2.legend()
    
    # Plot 3: Directional derivative at a specific point
    ax3 = fig.add_subplot(133)
    
    # Pick a point (current setup)
    px, py = 2.0, 1.0
    grad = grad_f(px, py)
    grad_mag = np.linalg.norm(grad)
    
    # Compute directional derivative for all directions
    angles = np.linspace(0, 2*np.pi, 100)
    dir_derivs = []
    for theta in angles:
        direction = np.array([np.cos(theta), np.sin(theta)])
        dir_deriv = np.dot(grad, direction)
        dir_derivs.append(dir_deriv)
    
    # Plot directional derivative vs angle
    ax3.plot(np.degrees(angles), dir_derivs, 'b-', linewidth=2)
    ax3.axhline(y=0, color='k', linewidth=0.5)
    ax3.axhline(y=grad_mag, color='g', linestyle='--', label=f'Max = |grad| = {grad_mag:.2f}')
    ax3.axhline(y=-grad_mag, color='r', linestyle='--', label=f'Min = -|grad| = {-grad_mag:.2f}')
    
    # Mark special directions
    grad_angle = np.degrees(np.arctan2(grad[1], grad[0]))
    ax3.axvline(x=grad_angle, color='g', alpha=0.5)
    ax3.axvline(x=grad_angle + 180, color='r', alpha=0.5)
    
    ax3.set_xlabel('Setup Change Direction (degrees)')
    ax3.set_ylabel('Rate of Lap Time Change')
    ax3.set_title(f'Sensitivity vs Direction at Setup ({px}, {py})\nWhich way to adjust?')
    ax3.legend(loc='lower right')
    ax3.set_xticks([0, 90, 180, 270, 360])
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"At setup (wing={px}, height={py}):")
    print(f"  Gradient = {grad}")
    print(f"  Gradient magnitude = {grad_mag:.2f}")
    print(f"  Gradient direction = {grad_angle:.1f} degrees")
    print(f"\n  To make car SLOWER fastest: change setup at {grad_angle:.1f} degrees (with gradient)")
    print(f"  To make car FASTER fastest: change setup at {grad_angle + 180:.1f} degrees (against gradient)")
    print(f"  To trade off (same lap time): change at {grad_angle + 90:.1f} degrees or {grad_angle - 90:.1f} degrees")

visualize_gradient_directions()

In [ ]:
# Visualize a 2D function and its partial derivatives
# F1: Lap time as a function of two setup parameters
def lap_time_surface(wing_angle, ride_height):
    return wing_angle**2 + ride_height**2

# Create meshgrid
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)
Z = lap_time_surface(X, Y)

fig = plt.figure(figsize=(15, 5))

# 3D surface
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.8)
ax1.set_xlabel('Wing Angle')
ax1.set_ylabel('Ride Height')
ax1.set_zlabel('Lap Time')
ax1.set_title('Lap Time = wing² + height²')

# Contour plot with gradient vectors
ax2 = fig.add_subplot(132)
contour = ax2.contour(X, Y, Z, levels=15, cmap=cm.viridis)
ax2.clabel(contour, inline=True, fontsize=8)

# Add gradient vectors at some setup points
setup_points = [(-2, -2), (-2, 0), (0, 2), (1, 1), (2, -1)]
for px, py in setup_points:
    grad_x = 2 * px  # d(lap_time)/d(wing_angle) = 2*wing_angle
    grad_y = 2 * py  # d(lap_time)/d(ride_height) = 2*ride_height
    ax2.arrow(px, py, grad_x*0.3, grad_y*0.3, head_width=0.15, head_length=0.1, fc='red', ec='red')

ax2.set_xlabel('Wing Angle')
ax2.set_ylabel('Ride Height')
ax2.set_title('Setup Space with Gradient Vectors\n(Red arrows = direction of increasing lap time)')
ax2.set_aspect('equal')

# Slice at ride_height=1 — what happens when we only vary wing angle?
ax3 = fig.add_subplot(133)
height_fixed = 1
z_slice = lap_time_surface(x, height_fixed)
ax3.plot(x, z_slice, 'b-', linewidth=2)
ax3.set_xlabel('Wing Angle')
ax3.set_ylabel(f'Lap Time (height={height_fixed})')
ax3.set_title(f'Partial View: Vary Wing Only (height={height_fixed})\nd(lap_time)/d(wing) = 2*wing')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 3. The Gradient

The **gradient** is the vector of all partial derivatives:

$$\nabla f = \begin{bmatrix} \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_n} \end{bmatrix}$$

**F1 analogy:** The gradient is the engineer's **complete sensitivity report**. Instead of asking about one parameter at a time, the gradient bundles ALL partial derivatives into a single vector: "Here's how sensitive lap time is to wing angle, ride height, spring stiffness, tire pressure... all at once." It tells you which parameter to change to get the biggest improvement, and by how much.

### Key Properties

1. **Direction**: Points in the direction of steepest **increase** (toward slower lap times)
2. **Magnitude**: Tells how steep that increase is (how sensitive the car is to changes)
3. **To minimize**: Move in the **opposite** direction (negative gradient = toward faster lap times)

### Deep Dive: Why the Chain Rule is CRITICAL for Deep Learning

The chain rule is not just another calculus rule - it's the **mathematical heart of backpropagation**. Without it, we couldn't train neural networks.

**F1 analogy:** The chain rule describes **sequential dependencies** in a system. In F1, engine power affects wheel torque, which affects tire slip, which affects cornering speed, which affects lap time. If you want to know "how does a 5 HP increase in engine power affect lap time?", you need to chain together the sensitivities at each link: (d_torque/d_power) x (d_slip/d_torque) x (d_cornering/d_slip) x (d_laptime/d_cornering). Each link multiplies the effect through the chain — that's the chain rule.

#### The Core Insight

When you have nested functions (f composed with g), changes **propagate** through the chain:

$$\text{small change in } x \rightarrow \text{change in } g(x) \rightarrow \text{change in } f(g(x))$$

The chain rule says: **multiply the rates of change at each step**.

#### Breaking Down the Formula

For $y = f(g(x))$, let's call $u = g(x)$ (the intermediate value):

$$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$

| Component | Meaning | Neural Network Terms | F1 Analogy |
|-----------|---------|----------------------|------------|
| $\frac{du}{dx}$ | How much does u change when x changes? | "Local gradient" of layer | How torque changes with engine power |
| $\frac{dy}{du}$ | How much does y change when u changes? | "Upstream gradient" from later layers | How lap time changes with torque |
| $\frac{dy}{dx}$ | How much does y change when x changes? | "Full gradient" through the network | How lap time changes with engine power |

#### Visual: How Changes Propagate

```
Input x                        Output y
   |                              |
   v                              v
   x ----[g]----> u = g(x) ----[f]----> y = f(u)
   
   Δx    ->     Δu = (dg/dx)·Δx   ->   Δy = (df/du)·Δu
                                            = (df/du)·(dg/dx)·Δx
```

The change in x gets **amplified (or diminished)** at each step, and the total effect is the product!

**F1 analogy:** A small engine mapping change ($\Delta x$) produces a torque change, which produces a speed change, which produces a lap time change. If each step amplifies by 2x, the total effect is $2 \times 2 \times 2 = 8x$. But if one link attenuates (say tire slip absorbs the torque), the effect might be $2 \times 0.1 \times 2 = 0.4x$. This "vanishing" effect is exactly the **vanishing gradient problem** in deep networks.

In [ ]:
# Step-by-step example with ACTUAL NUMBERS
# F1 context: How engine mapping affects lap time through a chain of dependencies
# Let's trace through f(g(x)) = (2x + 1)³ at x = 2

print("=" * 60)
print("CHAIN RULE: Step-by-Step with Actual Numbers")
print("=" * 60)
print("\nFunction: y = (2x + 1)³")
print("This is f(g(x)) where g(x) = 2x + 1 and f(u) = u³")
print("\nF1 context: x = engine power setting, g(x) = wheel torque,")
print("f(g(x)) = effect on lap time (cubed relationship)")
print("\n" + "-" * 60)

x = 2
print(f"Evaluating at x = {x}")

# Step 1: Forward pass - compute intermediate and final values
u = 2*x + 1  # g(x)
y = u**3      # f(u)

print(f"\n1. FORWARD PASS:")
print(f"   u = g(x) = 2({x}) + 1 = {u}")
print(f"   y = f(u) = {u}³ = {y}")

# Step 2: Compute local derivatives
dg_dx = 2          # derivative of g(x) = 2x + 1 is 2
df_du = 3 * u**2   # derivative of f(u) = u³ is 3u²

print(f"\n2. LOCAL DERIVATIVES (at this point):")
print(f"   dg/dx = d(2x+1)/dx = 2")
print(f"   df/du = d(u³)/du = 3u² = 3({u})² = {df_du}")

# Step 3: Apply chain rule
dy_dx = df_du * dg_dx

print(f"\n3. CHAIN RULE:")
print(f"   dy/dx = (df/du) x (dg/dx)")
print(f"         = {df_du} x {dg_dx}")
print(f"         = {dy_dx}")

# Verify with the analytical derivative
# y = (2x+1)³, so dy/dx = 3(2x+1)² × 2 = 6(2x+1)²
dy_dx_analytical = 6 * (2*x + 1)**2
print(f"\n4. VERIFICATION:")
print(f"   Analytical formula: dy/dx = 6(2x+1)²")
print(f"   At x = {x}: dy/dx = 6({2*x+1})² = {dy_dx_analytical}")
print(f"   Match: {dy_dx == dy_dx_analytical}")

# What does this mean?
print(f"\n5. INTERPRETATION:")
print(f"   If we increase x by a tiny amount dx = 0.001:")
print(f"   y will increase by approximately {dy_dx} x 0.001 = {dy_dx * 0.001}")
print(f"   F1: A tiny engine power tweak propagates and amplifies through the chain!")

# Verify numerically
h = 0.001
y_original = (2*x + 1)**3
y_nudged = (2*(x + h) + 1)**3
actual_change = y_nudged - y_original
print(f"   Actual change: {y_nudged} - {y_original} = {actual_change:.6f}")
print(f"   Predicted change: {dy_dx * h:.6f}")

In [ ]:
# Visualization: Chain Rule as Signal Propagation
# Shows how a small change propagates through composed functions
# F1: Like tracing how an engine mapping change ripples through the drivetrain

def visualize_chain_rule_propagation():
    """
    Visualize how changes propagate through composed functions.
    f(g(x)) = sin(x²) 
    F1: Think of x as throttle input, g(x) = engine response, f(g(x)) = speed output
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    x = np.linspace(-2, 2, 200)
    
    # g(x) = x²  (engine response to throttle)
    g = x**2
    # f(u) = sin(u) where u = g(x)  (speed oscillation from engine response)
    f = np.sin(g)
    
    # Plot g(x)
    axes[0].plot(x, g, 'b-', linewidth=2)
    axes[0].set_xlabel('Throttle Input (x)')
    axes[0].set_ylabel('Engine Response u = g(x) = x²')
    axes[0].set_title('Step 1: Inner function\nThrottle -> Engine Response')
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(y=0, color='k', linewidth=0.5)
    axes[0].axvline(x=0, color='k', linewidth=0.5)
    
    # Highlight a point
    x0 = 1.5
    u0 = x0**2
    axes[0].scatter([x0], [u0], color='red', s=100, zorder=5)
    axes[0].annotate(f'x={x0}\nu={u0:.2f}', xy=(x0, u0), xytext=(x0+0.3, u0-0.5), fontsize=10)
    
    # Plot f(u) = sin(u)
    u = np.linspace(0, 4, 200)
    axes[1].plot(u, np.sin(u), 'g-', linewidth=2)
    axes[1].set_xlabel('Engine Response (u)')
    axes[1].set_ylabel('Speed Output y = f(u) = sin(u)')
    axes[1].set_title('Step 2: Outer function\nEngine Response -> Speed')
    axes[1].grid(True, alpha=0.3)
    axes[1].axhline(y=0, color='k', linewidth=0.5)
    
    y0 = np.sin(u0)
    axes[1].scatter([u0], [y0], color='red', s=100, zorder=5)
    axes[1].annotate(f'u={u0:.2f}\ny={y0:.2f}', xy=(u0, y0), xytext=(u0+0.3, y0+0.2), fontsize=10)
    
    # Plot the composition f(g(x))
    axes[2].plot(x, f, 'm-', linewidth=2)
    axes[2].set_xlabel('Throttle Input (x)')
    axes[2].set_ylabel('Speed Output y = sin(x²)')
    axes[2].set_title('Result: Full Chain\nThrottle -> Speed (composed)')
    axes[2].grid(True, alpha=0.3)
    axes[2].axhline(y=0, color='k', linewidth=0.5)
    axes[2].axvline(x=0, color='k', linewidth=0.5)
    
    axes[2].scatter([x0], [y0], color='red', s=100, zorder=5)
    axes[2].annotate(f'x={x0}\ny={y0:.2f}', xy=(x0, y0), xytext=(x0+0.2, y0+0.3), fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Now show the gradient computation
    print("\n" + "=" * 60)
    print("CHAIN RULE COMPUTATION for y = sin(x²) at x = 1.5")
    print("F1: How does a small throttle change affect final speed?")
    print("=" * 60)
    
    # At x = 1.5
    x_val = 1.5
    u_val = x_val**2
    y_val = np.sin(u_val)
    
    # Local gradients
    dg_dx = 2 * x_val          # d(x²)/dx = 2x
    df_du = np.cos(u_val)       # d(sin(u))/du = cos(u)
    
    # Chain rule
    dy_dx = df_du * dg_dx
    
    print(f"\n1. Forward pass (throttle -> engine -> speed):")
    print(f"   x (throttle) = {x_val}")
    print(f"   u (engine response) = x² = {u_val}")
    print(f"   y (speed) = sin(u) = {y_val:.4f}")
    
    print(f"\n2. Backward pass (computing sensitivities):")
    print(f"   dy/du = cos(u) = cos({u_val}) = {df_du:.4f}")
    print(f"   du/dx = 2x = 2({x_val}) = {dg_dx}")
    
    print(f"\n3. Chain rule (total sensitivity):")
    print(f"   dy/dx = (dy/du) x (du/dx)")
    print(f"         = {df_du:.4f} x {dg_dx}")
    print(f"         = {dy_dx:.4f}")
    
    # Verify numerically
    h = 1e-5
    numerical_grad = (np.sin((x_val + h)**2) - np.sin((x_val - h)**2)) / (2*h)
    print(f"\n4. Numerical verification: {numerical_grad:.4f}")

visualize_chain_rule_propagation()

### Computational Graphs: The Key to Backpropagation

A **computational graph** is a visual representation of how a function computes its output. Each node is an operation, and edges show data flow.

**Why does this matter?** Neural networks are just big computational graphs, and backpropagation is just the chain rule applied systematically through the graph!

**F1 analogy:** An F1 car's performance model is itself a computational graph. Inputs like fuel load, wing settings, and tire compound flow through physics equations (nodes) to produce outputs like predicted lap time. When the team wants to know "how does fuel load affect lap time?", they trace the graph backward — just like backpropagation.

#### Example: Computing gradients through a simple graph

Consider: $L = (wx + b - y)^2$ (squared error loss)

```
     w                              
     |                              
     v                              
x -->[ * ]--> z1 -->[ + ]--> z2 -->[ - ]--> z3 -->[ ² ]--> L
                      ^              ^
                      |              |
                      b              y (target)
```

**Forward pass (left to right):** Compute values at each node
**Backward pass (right to left):** Compute gradients using chain rule

In [ ]:
# Detailed walkthrough of forward and backward pass through computational graph
# L = (w*x + b - y)²
# F1 context: L = predicted lap time error, w = setup gain, x = parameter, b = baseline, y = target time

def computational_graph_example():
    """
    Step-by-step forward and backward pass through a computational graph.
    This is EXACTLY how neural network libraries compute gradients!
    
    F1 parallel: Like tracing how setup parameters flow through a 
    performance model to produce predicted lap time, then tracing back
    to find which parameter matters most.
    """
    print("=" * 70)
    print("COMPUTATIONAL GRAPH: L = (w*x + b - y)²")
    print("F1: Predicted lap time error from a simple car model")
    print("=" * 70)
    
    # Input values
    x = 2.0   # input (e.g., wing angle setting)
    y = 7.0   # target (e.g., target lap time)
    w = 3.0   # weight (e.g., sensitivity coefficient)
    b = 1.0   # bias (e.g., baseline performance)
    
    print(f"\nInputs: x={x} (wing angle), y={y} (target time), w={w} (sensitivity), b={b} (baseline)")
    print("\n" + "-" * 70)
    print("FORWARD PASS (compute values left to right)")
    print("-" * 70)
    
    # Forward pass - compute each node
    z1 = w * x          # multiplication
    print(f"z1 = w * x = {w} * {x} = {z1}")
    
    z2 = z1 + b         # addition
    print(f"z2 = z1 + b = {z1} + {b} = {z2}")
    
    z3 = z2 - y         # subtraction (error)
    print(f"z3 = z2 - y = {z2} - {y} = {z3}")
    
    L = z3 ** 2         # square (loss)
    print(f"L = z3² = {z3}² = {L}")
    
    print("\n" + "-" * 70)
    print("BACKWARD PASS (compute gradients right to left)")
    print("-" * 70)
    print("Starting from dL/dL = 1 (gradient of L with respect to itself)\n")
    
    # Backward pass - apply chain rule at each node
    
    # Start with gradient of loss w.r.t. itself
    dL_dL = 1
    print(f"dL/dL = {dL_dL}")
    
    # Node: L = z3² 
    # dL/dz3 = d(z3²)/dz3 = 2*z3
    dL_dz3 = dL_dL * (2 * z3)
    print(f"\nNode L = z3²:")
    print(f"  Local gradient: d(z3²)/dz3 = 2*z3 = 2*{z3} = {2*z3}")
    print(f"  dL/dz3 = dL/dL x d(z3²)/dz3 = {dL_dL} x {2*z3} = {dL_dz3}")
    
    # Node: z3 = z2 - y
    # dz3/dz2 = 1, dz3/dy = -1
    dL_dz2 = dL_dz3 * 1
    dL_dy = dL_dz3 * (-1)
    print(f"\nNode z3 = z2 - y:")
    print(f"  Local gradients: dz3/dz2 = 1, dz3/dy = -1")
    print(f"  dL/dz2 = dL/dz3 x 1 = {dL_dz3} x 1 = {dL_dz2}")
    print(f"  dL/dy = dL/dz3 x (-1) = {dL_dz3} x (-1) = {dL_dy}")
    
    # Node: z2 = z1 + b
    # dz2/dz1 = 1, dz2/db = 1
    dL_dz1 = dL_dz2 * 1
    dL_db = dL_dz2 * 1
    print(f"\nNode z2 = z1 + b:")
    print(f"  Local gradients: dz2/dz1 = 1, dz2/db = 1")
    print(f"  dL/dz1 = dL/dz2 x 1 = {dL_dz2} x 1 = {dL_dz1}")
    print(f"  dL/db = dL/dz2 x 1 = {dL_dz2} x 1 = {dL_db}")
    
    # Node: z1 = w * x
    # dz1/dw = x, dz1/dx = w
    dL_dw = dL_dz1 * x
    dL_dx = dL_dz1 * w
    print(f"\nNode z1 = w * x:")
    print(f"  Local gradients: dz1/dw = x = {x}, dz1/dx = w = {w}")
    print(f"  dL/dw = dL/dz1 x x = {dL_dz1} x {x} = {dL_dw}")
    print(f"  dL/dx = dL/dz1 x w = {dL_dz1} x {w} = {dL_dx}")
    
    print("\n" + "-" * 70)
    print("SUMMARY OF GRADIENTS")
    print("-" * 70)
    print(f"dL/dw = {dL_dw}  <- How much changing sensitivity coefficient affects error")
    print(f"dL/db = {dL_db}  <- How much changing baseline affects error")
    print(f"dL/dx = {dL_dx}  <- How much changing wing angle affects error")
    
    print("\n" + "-" * 70)
    print("VERIFICATION with numerical gradients")
    print("-" * 70)
    h = 1e-5
    
    loss = lambda w, b, x, y: (w*x + b - y)**2
    
    dL_dw_num = (loss(w+h, b, x, y) - loss(w-h, b, x, y)) / (2*h)
    dL_db_num = (loss(w, b+h, x, y) - loss(w, b-h, x, y)) / (2*h)
    
    print(f"dL/dw: analytical = {dL_dw}, numerical = {dL_dw_num:.4f}")
    print(f"dL/db: analytical = {dL_db}, numerical = {dL_db_num:.4f}")
    
    return dL_dw, dL_db

dL_dw, dL_db = computational_graph_example()

### Deep Dive: Understanding Gradient Descent

Gradient descent is the **optimization engine** of deep learning. Let's build deep intuition.

#### The Core Idea in Plain English

You're lost on a foggy mountainside and want to reach the lowest valley. What do you do?
1. **Feel the slope** under your feet (compute gradient)
2. **Take a step downhill** (move opposite to gradient)
3. **Repeat** until you reach flat ground (gradient is zero)

That's gradient descent!

**F1 analogy:** Think of this as **iterative car setup optimization**. Your team arrives at a new circuit on Friday morning with a baseline setup. Each practice session is a gradient descent step:
1. **Run the car** and collect data (compute loss)
2. **Analyze which setup changes** would improve lap time most (compute gradient)
3. **Make the changes** in the direction of improvement (update parameters)
4. **Repeat** until you've found the optimal setup (converged)

The team can't test every possible setup combination — there are too many parameters. Instead, they iteratively move "downhill" in setup space toward the fastest lap time.

#### The Update Rule Decoded

$$\theta_{\text{new}} = \theta_{\text{old}} - \alpha \nabla L(\theta)$$

| Component | Meaning | Mountain Analogy | F1 Analogy |
|-----------|---------|------------------|------------|
| $\theta$ | Parameters (weights) | Your position on the mountain | Current car setup |
| $\nabla L(\theta)$ | Gradient of loss | Which way is uphill | Which setup direction makes the car slower |
| $-\nabla L(\theta)$ | Negative gradient | Which way is downhill | Which setup direction makes the car faster |
| $\alpha$ | Learning rate | Size of your steps | How aggressive the setup changes are |
| $\alpha \nabla L(\theta)$ | The actual step | How far you move | The actual setup adjustment made |

#### The Learning Rate $\alpha$ is Critical

| Learning rate | What happens | Problem | F1 Parallel |
|---------------|--------------|---------|-------------|
| **Too small** | Tiny steps, very slow progress | Takes forever to converge | Conservative engineer: 0.1-degree wing changes, wastes practice sessions |
| **Too large** | Big steps, overshoots minimum | Oscillates or diverges | Aggressive engineer: 5-degree wing changes, car oscillates between too much/too little downforce |
| **Just right** | Steady progress, converges | Sweet spot (hard to find!) | Experienced engineer: right-sized changes that converge to optimal setup |

This is why learning rate scheduling and adaptive optimizers (Adam) are important in practice.

In [ ]:
def compute_gradient(f, point, h=1e-5):
    """Compute gradient of f at point using numerical differentiation.
    
    F1 context: Compute the full sensitivity vector — how does lap time
    respond to changes in each setup parameter?
    """
    point = np.array(point, dtype=float)
    grad = np.zeros_like(point)
    for i in range(len(point)):
        grad[i] = partial_derivative(f, point, i, h)
    return grad

# Example: lap_time(wing, height) = wing² + height²
# The optimal setup is at (0, 0)
def lap_time(p):
    return p[0]**2 + p[1]**2

setup = np.array([3.0, 4.0])
grad = compute_gradient(lap_time, setup)

print(f"Current setup (wing={setup[0]}, height={setup[1]}):")
print(f"  Lap time = {lap_time(setup)}")
print(f"  Gradient (sensitivity) = {grad}")
print(f"  Gradient magnitude = {np.linalg.norm(grad):.4f}")
print(f"\nTo improve lap time, adjust setup by: {-grad}")
print("(Move opposite to gradient = toward faster lap times)")

In [ ]:
# Visualize gradient field over the setup landscape
# F1: Each arrow shows which direction makes the car SLOWER at that setup point
def lap_time_2d(wing, height):
    return wing**2 + height**2

# Create grid of setup combinations
wing_range = np.linspace(-3, 3, 15)
height_range = np.linspace(-3, 3, 15)
W, H = np.meshgrid(wing_range, height_range)

# Compute gradient at each setup point
U = 2 * W  # d(lap_time)/d(wing)
V = 2 * H  # d(lap_time)/d(height)

# Normalize for better visualization
magnitude = np.sqrt(U**2 + V**2)
U_norm = U / (magnitude + 1e-10)
V_norm = V / (magnitude + 1e-10)

plt.figure(figsize=(10, 8))

# Contour plot (iso-lap-time lines)
wing_fine = np.linspace(-3, 3, 100)
height_fine = np.linspace(-3, 3, 100)
W_fine, H_fine = np.meshgrid(wing_fine, height_fine)
Z_fine = lap_time_2d(W_fine, H_fine)
plt.contour(W_fine, H_fine, Z_fine, levels=15, cmap=cm.viridis, alpha=0.5)

# Gradient vectors (pointing toward slower lap times)
plt.quiver(W, H, U_norm, V_norm, magnitude, cmap=cm.Reds, alpha=0.8)

plt.xlabel('Wing Angle')
plt.ylabel('Ride Height')
plt.title('Gradient Field: Setup Sensitivity Landscape\nArrows point toward SLOWER lap times — go OPPOSITE for faster!')
plt.colorbar(label='Gradient magnitude (sensitivity)')
plt.axis('equal')
plt.show()

print("Notice: Gradients point away from the optimal setup (origin)")
print("To find the fastest setup, follow the NEGATIVE gradient (opposite direction)")

---

## 4. The Chain Rule

The **chain rule** is the foundation of backpropagation. It tells us how to differentiate composite functions.

### Single Variable

If $y = f(g(x))$, then:

$$\frac{dy}{dx} = \frac{df}{dg} \cdot \frac{dg}{dx}$$

### Intuition

If $x$ changes by a small amount $\Delta x$:
- $g$ changes by $\frac{dg}{dx} \cdot \Delta x$
- This causes $f$ to change by $\frac{df}{dg} \cdot (\frac{dg}{dx} \cdot \Delta x)$

The changes **multiply** through the chain!

**F1 analogy:** The chain rule is how F1 teams trace the effect of a low-level change (like fuel mixture) through a sequence of dependencies: fuel mixture affects combustion temperature, which affects power output, which affects wheel torque, which affects lap time. Each link has its own sensitivity, and the total effect is the product of all of them.

In [ ]:
# Example: y = (3x + 2)²
# F1: Like "lap time = (3*fuel_load + baseline)²"
# Let g(x) = 3x + 2, f(g) = g²
# dy/dx = df/dg * dg/dx = 2g * 3 = 6(3x + 2)

def y(x):
    return (3*x + 2)**2

def dy_dx_analytical(x):
    return 6 * (3*x + 2)

fuel_load = 1.0
print(f"y = (3x + 2)² at x = {fuel_load}")
print(f"  (F1: Lap time model with fuel_load = {fuel_load})")
print(f"y({fuel_load}) = {y(fuel_load)}")
print(f"dy/dx numerical:  {numerical_derivative(y, fuel_load):.6f}")
print(f"dy/dx analytical: {dy_dx_analytical(fuel_load):.6f}")
print(f"\nF1 interpretation: At fuel_load={fuel_load}, each unit of fuel changes")
print(f"lap time by {dy_dx_analytical(fuel_load):.1f} units (chain rule in action)")

In [ ]:
# Comprehensive visualization of gradient descent behavior
# F1: Like watching the team iterate on car setup across practice sessions
# Shows the path, learning rate effects, and convergence

def visualize_gradient_descent_comprehensive():
    """
    Create a comprehensive visualization showing:
    1. 3D view of the lap time surface with setup optimization path
    2. Top-down view (contour) with path
    3. Lap time over iterations
    4. Effect of different learning rates (setup aggressiveness)
    """
    
    # Define a lap time landscape: f(wing,height) = wing² + 10*height² (elongated bowl)
    # Height is more sensitive than wing — common in real F1 setups
    def f(p):
        return p[0]**2 + 10*p[1]**2
    
    def grad_f(p):
        return np.array([2*p[0], 20*p[1]])
    
    def gradient_descent(start, lr, n_steps):
        point = np.array(start, dtype=float)
        history = [point.copy()]
        for _ in range(n_steps):
            point = point - lr * grad_f(point)
            history.append(point.copy())
        return np.array(history)
    
    # Run setup optimization with different "aggressiveness" levels
    start = [3.0, 1.0]  # Initial setup: wing=3, height=1
    n_steps = 30
    
    lr_conservative = 0.01   # Very cautious engineer
    lr_good = 0.05           # Experienced engineer
    lr_aggressive = 0.09     # Bold engineer
    lr_reckless = 0.11       # Too aggressive — overshoots!
    
    hist_conservative = gradient_descent(start, lr_conservative, n_steps)
    hist_good = gradient_descent(start, lr_good, n_steps)
    hist_aggressive = gradient_descent(start, lr_aggressive, n_steps)
    hist_reckless = gradient_descent(start, lr_reckless, n_steps)
    
    # Create figure
    fig = plt.figure(figsize=(16, 10))
    
    # Create grid for surface plots
    x = np.linspace(-4, 4, 100)
    y = np.linspace(-2, 2, 100)
    X, Y = np.meshgrid(x, y)
    Z = X**2 + 10*Y**2
    
    # Plot 1: 3D surface with setup optimization path
    ax1 = fig.add_subplot(221, projection='3d')
    ax1.plot_surface(X, Y, Z, cmap=cm.viridis, alpha=0.6)
    
    # Add optimization path on surface
    path_z = [f(p) for p in hist_good]
    ax1.plot(hist_good[:, 0], hist_good[:, 1], path_z, 'r.-', 
             markersize=8, linewidth=2, label='Setup path')
    ax1.scatter([start[0]], [start[1]], [f(start)], color='green', s=100, marker='o')
    ax1.scatter([0], [0], [0], color='red', s=100, marker='*')
    
    ax1.set_xlabel('Wing Angle')
    ax1.set_ylabel('Ride Height')
    ax1.set_zlabel('Lap Time')
    ax1.set_title('3D View: Setup Optimization Path\n(Learning rate = 0.05)')
    
    # Plot 2: Contour with all paths
    ax2 = fig.add_subplot(222)
    contour = ax2.contour(X, Y, Z, levels=20, cmap=cm.viridis)
    ax2.clabel(contour, inline=True, fontsize=8)
    
    ax2.plot(hist_conservative[:, 0], hist_conservative[:, 1], 'b.-', markersize=5, 
             linewidth=1.5, label=f'lr={lr_conservative} (conservative)')
    ax2.plot(hist_good[:, 0], hist_good[:, 1], 'g.-', markersize=5, 
             linewidth=1.5, label=f'lr={lr_good} (experienced)')
    ax2.plot(hist_aggressive[:, 0], hist_aggressive[:, 1], 'orange', marker='.', markersize=5, 
             linewidth=1.5, label=f'lr={lr_aggressive} (aggressive)')
    ax2.plot(hist_reckless[:, 0], hist_reckless[:, 1], 'r.-', markersize=5, 
             linewidth=1.5, label=f'lr={lr_reckless} (reckless)')
    
    ax2.scatter([start[0]], [start[1]], color='green', s=150, marker='o', zorder=5, label='Friday Baseline')
    ax2.scatter([0], [0], color='red', s=150, marker='*', zorder=5, label='Optimal Setup')
    
    ax2.set_xlabel('Wing Angle')
    ax2.set_ylabel('Ride Height')
    ax2.set_title('Top View: Different Engineer Strategies')
    ax2.legend(loc='upper right', fontsize=9)
    ax2.set_aspect('equal')
    
    # Plot 3: Lap time curves
    ax3 = fig.add_subplot(223)
    
    losses_conservative = [f(p) for p in hist_conservative]
    losses_good = [f(p) for p in hist_good]
    losses_aggressive = [f(p) for p in hist_aggressive]
    losses_reckless = [f(p) for p in hist_reckless]
    
    ax3.plot(losses_conservative, 'b-', linewidth=2, label=f'lr={lr_conservative}')
    ax3.plot(losses_good, 'g-', linewidth=2, label=f'lr={lr_good}')
    ax3.plot(losses_aggressive, color='orange', linewidth=2, label=f'lr={lr_aggressive}')
    ax3.plot(losses_reckless, 'r-', linewidth=2, label=f'lr={lr_reckless}')
    
    ax3.set_xlabel('Practice Session (Iteration)')
    ax3.set_ylabel('Lap Time')
    ax3.set_title('Lap Time Improvement Over Sessions')
    ax3.legend()
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Zoomed in first few steps
    ax4 = fig.add_subplot(224)
    
    # Show step-by-step for good learning rate
    for i in range(min(8, len(hist_good)-1)):
        p1 = hist_good[i]
        p2 = hist_good[i+1]
        grad = grad_f(p1)
        
        # Point
        ax4.scatter([p1[0]], [p1[1]], color='blue', s=60, zorder=5)
        ax4.annotate(f'{i}', xy=(p1[0], p1[1]), xytext=(p1[0]+0.1, p1[1]+0.1), fontsize=9)
        
        # Gradient (scaled for visualization)
        ax4.arrow(p1[0], p1[1], -grad[0]*0.02, -grad[1]*0.02, 
                 head_width=0.05, head_length=0.02, fc='red', ec='red', alpha=0.5)
        
        # Actual step
        ax4.arrow(p1[0], p1[1], (p2[0]-p1[0])*0.95, (p2[1]-p1[1])*0.95,
                 head_width=0.05, head_length=0.02, fc='green', ec='green')
    
    contour2 = ax4.contour(X, Y, Z, levels=20, cmap=cm.viridis, alpha=0.5)
    ax4.set_xlabel('Wing Angle')
    ax4.set_ylabel('Ride Height')
    ax4.set_title('Step-by-Step Setup Changes (lr=0.05)\nGreen: actual changes, Red: gradient direction')
    ax4.set_xlim([-1, 4])
    ax4.set_ylim([-0.5, 1.5])
    
    plt.tight_layout()
    plt.show()
    
    print("Key observations (F1 setup optimization):")
    print(f"- lr={lr_conservative} (conservative): Too cautious — wastes practice sessions")
    print(f"- lr={lr_good} (experienced): Efficient — reaches optimal setup quickly")
    print(f"- lr={lr_aggressive} (aggressive): Oscillates but finds the neighborhood")
    print(f"- lr={lr_reckless} (reckless): Overshoots wildly — car gets worse before better!")

visualize_gradient_descent_comprehensive()

### The Local Minima Problem

Real loss surfaces are rarely simple bowls. They often have:
- **Local minima**: Points that look like minima locally but aren't the global best
- **Saddle points**: Points where gradient is zero but it's neither min nor max
- **Plateaus**: Flat regions where gradient is tiny

**F1 analogy:** This is the classic "good enough" setup trap. Your team finds a setup that feels fast — the car is balanced, the driver is happy, and small tweaks make it slower. But there might be a completely different setup philosophy (say, switching from high-downforce to low-downforce) that's actually faster overall. You've found a **local minimum** — a setup that's locally optimal but not globally. Getting out requires a big, bold change (like momentum in gradient descent) or starting from a different baseline entirely.

Neural networks have extremely complex loss landscapes. Fortunately:
1. In high dimensions, true local minima are rare (saddle points are more common)
2. Many local minima have similar loss values
3. Modern optimizers (Adam, etc.) can escape shallow local minima

In [ ]:
# Visualize local minima, saddle points, and the challenges they pose
# F1: Multiple "fast" setups exist — but which is THE fastest?

def visualize_local_minima_problem():
    """
    Visualize a lap time landscape with multiple local minima
    and show how gradient descent can get stuck in a "good enough" setup.
    """
    
    # Create a function with multiple local minima
    # f(x) = sin(x) + 0.1*x² (creates multiple valleys — multiple "fast" setups)
    def f_1d(x):
        return np.sin(3*x) + 0.1*x**2
    
    def df_1d(x):
        return 3*np.cos(3*x) + 0.2*x
    
    # 1D visualization
    fig, axes = plt.subplots(1, 3, figsize=(16, 4))
    
    x = np.linspace(-4, 4, 200)
    y = f_1d(x)
    
    axes[0].plot(x, y, 'b-', linewidth=2)
    axes[0].set_xlabel('Setup Parameter')
    axes[0].set_ylabel('Lap Time')
    axes[0].set_title('Lap Time Landscape with Multiple "Fast" Setups')
    axes[0].grid(True, alpha=0.3)
    
    # Mark local minima (where derivative crosses zero from - to +)
    for xi in np.linspace(-4, 4, 1000):
        if abs(df_1d(xi)) < 0.05 and f_1d(xi-0.01) > f_1d(xi) < f_1d(xi+0.01):
            axes[0].scatter([xi], [f_1d(xi)], color='red', s=100, marker='v', zorder=5)
    
    axes[0].annotate('Fastest\nsetup', xy=(-2.1, f_1d(-2.1)), xytext=(-3, 1),
                    arrowprops=dict(arrowstyle='->', color='green'), fontsize=10, color='green')
    axes[0].annotate('"Good enough"\nsetup', xy=(0.0, f_1d(0.0)), xytext=(1, 1.5),
                    arrowprops=dict(arrowstyle='->', color='red'), fontsize=10, color='red')
    
    # Run GD from different starting setups
    def gd_1d(x0, lr=0.1, n_steps=50):
        x = x0
        history = [x]
        for _ in range(n_steps):
            x = x - lr * df_1d(x)
            history.append(x)
        return np.array(history)
    
    starts = [-3.5, -1.0, 1.5, 3.0]
    colors = ['green', 'red', 'orange', 'purple']
    
    axes[1].plot(x, y, 'b-', linewidth=2, alpha=0.5)
    for start, color in zip(starts, colors):
        hist = gd_1d(start, lr=0.05, n_steps=100)
        y_hist = f_1d(hist)
        axes[1].plot(hist, y_hist, '.-', color=color, markersize=4, 
                    linewidth=1, label=f'Start={start}')
        axes[1].scatter([start], [f_1d(start)], color=color, s=100, marker='o', zorder=5)
    
    axes[1].set_xlabel('Setup Parameter')
    axes[1].set_ylabel('Lap Time')
    axes[1].set_title('Optimization from Different Friday Baselines')
    axes[1].legend(fontsize=9)
    axes[1].grid(True, alpha=0.3)
    
    # Show final lap times
    final_times = []
    for start in starts:
        hist = gd_1d(start, lr=0.05, n_steps=100)
        final_times.append(f_1d(hist[-1]))
    
    axes[2].bar(range(len(starts)), final_times, color=colors)
    axes[2].set_xticks(range(len(starts)))
    axes[2].set_xticklabels([f'Start={s}' for s in starts])
    axes[2].set_xlabel('Friday Baseline Setup')
    axes[2].set_ylabel('Final Lap Time')
    axes[2].set_title('Final Lap Time Depends on Starting Setup!\n(Different baselines = different local optima)')
    axes[2].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("Key insight: Gradient descent finds LOCAL optima, not necessarily the GLOBAL optimum.")
    print("The final setup depends on where you started!")
    print("\nF1 strategies to escape local minima:")
    print("1. Try multiple baseline setups on Friday (multiple random starts)")
    print("2. Use momentum to 'carry speed' past shallow local minima")
    print("3. Add randomness (SGD noise = trying random setup experiments)")
    print("4. Use learning rate schedules (big changes early, fine-tuning later)")

visualize_local_minima_problem()

In [ ]:
# Visualize saddle points in 2D
# F1: A setup where one parameter is optimized but another isn't —
# e.g., wing angle is perfect but ride height is wrong

def visualize_saddle_point():
    """
    Visualize a saddle point and why it's problematic for gradient descent.
    F1: Like a setup that's optimal for straight-line speed but terrible for corners.
    """
    # Classic saddle: f(x,y) = x² - y²
    def f_saddle(x, y):
        return x**2 - y**2
    
    def grad_saddle(p):
        return np.array([2*p[0], -2*p[1]])
    
    fig = plt.figure(figsize=(16, 5))
    
    # Create grid
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-2, 2, 100)
    X, Y = np.meshgrid(x, y)
    Z = f_saddle(X, Y)
    
    # 3D surface
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha=0.8)
    ax1.scatter([0], [0], [0], color='black', s=200, marker='o')
    ax1.set_xlabel('Wing Angle')
    ax1.set_ylabel('Ride Height')
    ax1.set_zlabel('Lap Time')
    ax1.set_title('Saddle Point: f = wing² - height²\nOptimal in one dimension, not the other')
    
    # Contour plot
    ax2 = fig.add_subplot(132)
    contour = ax2.contour(X, Y, Z, levels=20, cmap=cm.coolwarm)
    ax2.clabel(contour, inline=True, fontsize=8)
    ax2.scatter([0], [0], color='black', s=200, marker='o', label='Saddle point')
    
    # Draw gradient arrows around saddle point
    for px, py in [(0.5, 0), (-0.5, 0), (0, 0.5), (0, -0.5)]:
        grad = grad_saddle([px, py])
        ax2.arrow(px, py, -grad[0]*0.15, -grad[1]*0.15, head_width=0.05, 
                 head_length=0.02, fc='green', ec='green')
    
    ax2.set_xlabel('Wing Angle')
    ax2.set_ylabel('Ride Height')
    ax2.set_title('Setup Space Contours\nGreen arrows: negative gradient direction')
    ax2.legend()
    ax2.set_aspect('equal')
    
    # 1D slices through saddle point
    ax3 = fig.add_subplot(133)
    x_slice = np.linspace(-2, 2, 100)
    ax3.plot(x_slice, x_slice**2, 'b-', linewidth=2, label='Vary wing only: minimum')
    ax3.plot(x_slice, -x_slice**2, 'r-', linewidth=2, label='Vary height only: maximum')
    ax3.axhline(y=0, color='k', linewidth=0.5)
    ax3.axvline(x=0, color='k', linewidth=0.5)
    ax3.scatter([0], [0], color='black', s=100, zorder=5)
    ax3.set_xlabel('Parameter Value')
    ax3.set_ylabel('Lap Time')
    ax3.set_title('1D Slices Through Saddle\nSame point is min AND max!')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("At a saddle point:")
    print("- Gradient is zero (looks like an optimum)")
    print("- But it's a minimum in some directions, maximum in others")
    print("- GD can get stuck here if approaching from certain directions")
    print("\nF1 analogy: A setup where wing angle is perfect but ride height")
    print("is all wrong. The engineer who only looks at wing data thinks")
    print("the car is optimized — but changing ride height would unlock more pace!")
    print("\nIn high-dimensional neural network loss landscapes:")
    print("- Saddle points are MUCH more common than local minima")
    print("- Momentum helps escape saddle points by building up velocity")

visualize_saddle_point()

### Chain Rule in Neural Networks

Consider a simple network:

$$\text{Input } x \rightarrow z = wx + b \rightarrow a = \sigma(z) \rightarrow L = (a - y)^2$$

To find $\frac{\partial L}{\partial w}$, we apply the chain rule:

$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial a} \cdot \frac{\partial a}{\partial z} \cdot \frac{\partial z}{\partial w}$$

**F1 analogy:** This is like tracing: "How does changing a suspension spring rate (w) affect the ride dynamics (z), which affects the tire contact patch (a), which affects the lap time (L)?" Each link has its own sensitivity, and we multiply them all together to get the total effect of the spring rate on lap time.

In [ ]:
# Computational graph example
# Forward pass: x -> z = wx + b -> a = sigmoid(z) -> L = (a - y)²
# F1: wing_angle -> aero_force = w*wing + bias -> grip = sigmoid(aero) -> lap_time_error

def forward_and_backward(x, y_true, w, b):
    """Compute forward pass and gradients using chain rule.
    
    F1 parallel: Trace how a setup parameter flows through a performance
    model and back again to find sensitivities.
    """
    
    # Forward pass
    z = w * x + b
    a = sigmoid(z)
    L = (a - y_true)**2
    
    print("=== Forward Pass (Setup -> Performance -> Lap Time Error) ===")
    print(f"x (wing angle) = {x}")
    print(f"z = w*x + b = {w}*{x} + {b} = {z}")
    print(f"a = sigmoid(z) = {a:.6f}")
    print(f"L = (a - y_target)² = ({a:.6f} - {y_true})² = {L:.6f}")
    
    # Backward pass (chain rule)
    print("\n=== Backward Pass (Chain Rule: Tracing Sensitivities Back) ===")
    
    # dL/da
    dL_da = 2 * (a - y_true)
    print(f"dL/da = 2(a - y) = {dL_da:.6f}")
    
    # da/dz (sigmoid derivative)
    da_dz = a * (1 - a)
    print(f"da/dz = sigmoid(z)(1 - sigmoid(z)) = {da_dz:.6f}")
    
    # dz/dw
    dz_dw = x
    print(f"dz/dw = x = {dz_dw}")
    
    # dz/db
    dz_db = 1
    print(f"dz/db = 1")
    
    # Chain rule
    dL_dz = dL_da * da_dz
    dL_dw = dL_dz * dz_dw
    dL_db = dL_dz * dz_db
    
    print(f"\ndL/dw = dL/da * da/dz * dz/dw = {dL_dw:.6f}")
    print(f"dL/db = dL/da * da/dz * dz/db = {dL_db:.6f}")
    
    return L, dL_dw, dL_db

# Example
x = 2.0
y_true = 1.0
w = 0.5
b = 0.1

L, dL_dw, dL_db = forward_and_backward(x, y_true, w, b)

In [ ]:
# Verify with numerical gradient — the "sanity check" every engineer should do
h = 1e-5

def loss(w, b, x=2.0, y=1.0):
    z = w * x + b
    a = sigmoid(z)
    return (a - y)**2

# Numerical gradients
dL_dw_numerical = (loss(w + h, b) - loss(w - h, b)) / (2 * h)
dL_db_numerical = (loss(w, b + h) - loss(w, b - h)) / (2 * h)

print("Verification with numerical gradients (the engineer's double-check):")
print(f"dL/dw: analytical = {dL_dw:.6f}, numerical = {dL_dw_numerical:.6f}")
print(f"dL/db: analytical = {dL_db:.6f}, numerical = {dL_db_numerical:.6f}")

### Multivariate Chain Rule

When a variable affects the output through multiple paths:

$$\frac{\partial L}{\partial x} = \sum_{i} \frac{\partial L}{\partial y_i} \cdot \frac{\partial y_i}{\partial x}$$

This is why we **sum** gradients when a variable is used multiple times.

**F1 analogy:** Tire pressure affects lap time through multiple paths simultaneously — it changes both straight-line grip AND cornering grip AND tire wear rate. The total effect of tire pressure on lap time is the **sum** of its effects through each path. This is the multivariate chain rule in action.

In [ ]:
# Example: f = x*y + x*z where y and z both depend on x
# F1: Tire pressure (x) affects cornering (x*y path) AND straight-line (x*z path)
# Actually, let's do: f(x) = x² + x (x is used twice)

# Computational graph:
# x --> a = x  --\
#                 +--> c = a * b --> f = c + d
# x --> b = x  --/                      |
#                                       |
# x --> d = x  -------------------------/

# This is: f = x*x + x = x² + x
# df/dx = 2x + 1 (by calculus)

# But through the graph:
# df/dx = df/dc * dc/da * da/dx + df/dc * dc/db * db/dx + df/dd * dd/dx
#       = 1 * b * 1 + 1 * a * 1 + 1 * 1
#       = x + x + 1 = 2x + 1

tire_pressure = 3.0
print(f"f(x) = x² + x at x = {tire_pressure}")
print(f"F1: Total lap time effect when tire pressure appears in multiple paths")
print(f"f({tire_pressure}) = {tire_pressure**2 + tire_pressure}")
print(f"df/dx (analytical) = 2x + 1 = {2*tire_pressure + 1}")

# Through computational graph — summing all paths
a = tire_pressure
b = tire_pressure  
c = a * b  # = x² (cornering effect)
d = tire_pressure  # (straight-line effect)
f = c + d  # = x² + x (total)

# Backward — sum gradients from all paths
df_dc = 1
df_dd = 1
dc_da = b  # = x
dc_db = a  # = x
da_dx = 1
db_dx = 1
dd_dx = 1

# Sum all paths from f to x
df_dx = df_dc * dc_da * da_dx + df_dc * dc_db * db_dx + df_dd * dd_dx
print(f"df/dx (computational graph, summing paths) = {df_dx}")
print(f"\nKey: We SUM the contributions from each path x takes through the graph")

---

## 5. Gradient Descent

**Gradient descent** is the optimization algorithm that powers deep learning:

$$\theta_{\text{new}} = \theta_{\text{old}} - \alpha \nabla L(\theta)$$

Where:
- $\theta$: Parameters (weights)
- $\alpha$: Learning rate (step size)
- $\nabla L$: Gradient of loss with respect to parameters

**F1 analogy:** This is the mathematical version of what every F1 team does during a race weekend. $\theta$ is the car setup. $L(\theta)$ is the lap time. $\nabla L$ tells you which parameters to change and by how much. $\alpha$ controls how aggressive those changes are. Each practice session is an iteration of gradient descent, moving the setup toward the fastest possible lap time.

In [ ]:
def gradient_descent_1d(f, df, x0, learning_rate=0.1, n_steps=50):
    """Gradient descent for 1D function.
    
    F1: Iteratively adjust a single setup parameter to minimize lap time.
    """
    x = x0
    history = [(x, f(x))]
    
    for i in range(n_steps):
        grad = df(x)
        x = x - learning_rate * grad
        history.append((x, f(x)))
        
    return x, history

# Minimize f(x) = (x - 3)² — optimal setup parameter is at x=3
# F1: "Find the optimal brake bias setting"
f = lambda x: (x - 3)**2
df = lambda x: 2 * (x - 3)

brake_bias_final, history = gradient_descent_1d(f, df, x0=10.0, learning_rate=0.1, n_steps=30)

print(f"Brake bias optimization:")
print(f"  Starting value: 10.0")
print(f"  Optimal found:  {brake_bias_final:.6f}")
print(f"  Lap time at optimum: {f(brake_bias_final):.6f}")
print(f"  True optimum: x = 3")

# Visualize
x_range = np.linspace(-2, 12, 100)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x_range, f(x_range), 'b-', linewidth=2, label='Lap time = (brake_bias - 3)²')
xs, ys = zip(*history)
plt.scatter(xs, ys, c=range(len(xs)), cmap='Reds', s=50, zorder=5)
plt.plot(xs, ys, 'r--', alpha=0.5)
plt.xlabel('Brake Bias Setting')
plt.ylabel('Lap Time Penalty')
plt.title('Gradient Descent: Finding Optimal Brake Bias')
plt.legend()
plt.colorbar(label='Iteration (Practice Run)')

plt.subplot(1, 2, 2)
plt.plot([h[1] for h in history], 'b-o')
plt.xlabel('Practice Run (Iteration)')
plt.ylabel('Lap Time Penalty')
plt.title('Lap Time Improvement Over Runs')

plt.tight_layout()
plt.show()

### 2D Gradient Descent

Now let's optimize two setup parameters simultaneously — this is where the gradient (not just the derivative) comes in. The gradient tells us the optimal combined direction to adjust **both** parameters at once.

In [ ]:
def gradient_descent_2d(f, grad_f, start, learning_rate=0.1, n_steps=50):
    """Gradient descent for 2D function.
    
    F1: Simultaneously optimize two setup parameters (e.g., wing angle and ride height).
    """
    point = np.array(start, dtype=float)
    history = [point.copy()]
    
    for i in range(n_steps):
        grad = grad_f(point)
        point = point - learning_rate * grad
        history.append(point.copy())
        
    return point, np.array(history)

# Minimize lap_time(wing, height) = wing² + height²
# Optimal setup at (0, 0)
def f(p):
    return p[0]**2 + p[1]**2

def grad_f(p):
    return np.array([2*p[0], 2*p[1]])

start = [4.0, 3.0]  # Friday morning baseline setup
final, history = gradient_descent_2d(f, grad_f, start, learning_rate=0.1, n_steps=30)

print(f"Friday baseline setup: wing={start[0]}, height={start[1]}")
print(f"Optimal setup found:   wing={final[0]:.6f}, height={final[1]:.6f}")
print(f"Final lap time: {f(final):.10f}")

# Visualize the setup optimization path
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=20, cmap=cm.viridis)
plt.plot(history[:, 0], history[:, 1], 'r.-', markersize=10, linewidth=2)
plt.scatter([start[0]], [start[1]], color='green', s=200, marker='o', label='Friday Baseline', zorder=5)
plt.scatter([final[0]], [final[1]], color='red', s=200, marker='*', label='Optimal Setup', zorder=5)
plt.xlabel('Wing Angle')
plt.ylabel('Ride Height')
plt.title('Gradient Descent: Finding Optimal Car Setup\nlap_time(wing, height) = wing² + height²')
plt.legend()
plt.colorbar(label='Lap Time')
plt.axis('equal')
plt.show()

### Effect of Learning Rate

**F1 analogy:** The learning rate is the engineer's **aggressiveness dial**. How big a setup change do you make between runs? Too small and you waste precious practice time making imperceptible changes. Too large and the car oscillates between being too stiff and too soft, never settling on the optimum.

### Calculus Concepts and Their ML Applications

| Calculus Concept | What it Means | ML Application | F1 Parallel |
|------------------|---------------|----------------|-------------|
| **Derivative** | Rate of change of output w.r.t. input | How loss changes when we change one weight | How lap time changes when we adjust wing angle |
| **Partial Derivative** | Rate of change w.r.t. one variable (others fixed) | Gradient component for one parameter | Effect of changing ONLY ride height on lap time |
| **Gradient** | Vector of all partial derivatives | Direction to update ALL weights at once | Complete sensitivity report for all setup parameters |
| **Chain Rule** | Derivative of composed functions = product of derivatives | Backpropagation through network layers | How engine power flows through drivetrain to affect speed |
| **Gradient Descent** | Iteratively move opposite to gradient | Core training algorithm for neural networks | Iterative car setup optimization across practice sessions |
| **Learning Rate** | Step size in gradient descent | Hyperparameter controlling training speed | How aggressive setup changes are between runs |
| **Local Minimum** | Point where gradient = 0 and function curves up | Where training might get stuck | A "good enough" setup that isn't actually the fastest |
| **Saddle Point** | Point where gradient = 0 but not min or max | Common in high-dim; momentum helps escape | Setup optimal in one dimension but not others |

### The Full Picture: How a Neural Network Learns

1. **Forward pass**: Input flows through network, computing activations layer by layer
2. **Loss computation**: Compare output to target, get a single number (the loss)
3. **Backward pass**: Use chain rule to compute gradient of loss w.r.t. every weight
4. **Parameter update**: Use gradient descent to update all weights
5. **Repeat**: Until loss is small enough

**F1 parallel:** This is exactly the race weekend workflow:
1. **Run the car** (forward pass through the physics of the track)
2. **Check the lap time** (compute the loss)
3. **Analyze telemetry** to find what to change (backward pass / gradient computation)
4. **Adjust the setup** (parameter update via gradient descent)
5. **Run again** (repeat until qualifying)

In [ ]:
# Compare different learning rates — the engineer's aggressiveness dial
learning_rates = [0.01, 0.1, 0.5, 0.95]
colors = ['blue', 'green', 'orange', 'red']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Contour plot with paths
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + Y**2

axes[0].contour(X, Y, Z, levels=20, cmap=cm.viridis, alpha=0.5)

for lr, color in zip(learning_rates, colors):
    final, history = gradient_descent_2d(f, grad_f, [4.0, 3.0], learning_rate=lr, n_steps=20)
    axes[0].plot(history[:, 0], history[:, 1], '.-', color=color, markersize=8, 
                 linewidth=2, label=f'lr={lr}')
    
    # Lap time curve
    losses = [f(p) for p in history]
    axes[1].plot(losses, color=color, linewidth=2, label=f'lr={lr}')

axes[0].set_xlabel('Wing Angle')
axes[0].set_ylabel('Ride Height')
axes[0].set_title('Setup Optimization Paths\n(Different Engineer Aggressiveness)')
axes[0].legend()
axes[0].axis('equal')

axes[1].set_xlabel('Practice Run (Iteration)')
axes[1].set_ylabel('Lap Time')
axes[1].set_title('Lap Time Convergence')
axes[1].legend()
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

print("F1 observations on learning rate (engineer aggressiveness):")
print("- Too small (0.01): Conservative — wastes practice sessions, slow to find pace")
print("- Good (0.1): Experienced — steady progress toward optimal setup")
print("- Larger (0.5): Bold — finds neighborhood fast but oscillates around it")
print("- Too large (0.95): Reckless — car swings wildly between extremes")

### A More Challenging Function: Rosenbrock

The Rosenbrock function is a classic optimization test:

$$f(x, y) = (1 - x)^2 + 100(y - x^2)^2$$

Minimum at $(1, 1)$. Famous for its narrow, curved valley.

**F1 analogy:** This is like a setup landscape with a narrow "sweet spot" — a long, winding valley of decent performance but only one truly optimal point. Think of it like the Monaco setup window: there's a narrow corridor of setups that work, and the optimal point requires precise tuning of correlated parameters (wing angle and ride height that depend on each other). This is why simple gradient descent struggles and why adaptive optimizers (like Adam in ML, or experienced engineers in F1) are so valuable.

In [ ]:
def rosenbrock(p):
    x, y = p
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(p):
    x, y = p
    dx = -2*(1 - x) - 400*x*(y - x**2)
    dy = 200*(y - x**2)
    return np.array([dx, dy])

# Visualize the Rosenbrock "Monaco setup landscape"
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=np.logspace(0, 3, 30), cmap=cm.viridis)
plt.scatter([1], [1], color='red', s=200, marker='*', label='Optimal Setup (1,1)', zorder=5)
plt.xlabel('Wing Angle')
plt.ylabel('Ride Height')
plt.title('The Rosenbrock "Monaco" Setup Landscape\nNotice the narrow curved valley — hard to optimize!')
plt.colorbar(label='Lap Time')
plt.legend()
plt.show()

In [ ]:
# Gradient descent on Rosenbrock (challenging! — like optimizing a Monaco setup)
start = [-1.0, 1.0]
final, history = gradient_descent_2d(rosenbrock, rosenbrock_grad, start, 
                                      learning_rate=0.001, n_steps=5000)

print(f"Starting setup: {start}")
print(f"Final setup:    [{final[0]:.4f}, {final[1]:.4f}]")
print(f"Lap time at final: {rosenbrock(final):.6f}")
print(f"True optimal setup: (1, 1), lap_time = 0")

# Visualize
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.contour(X, Y, Z, levels=np.logspace(0, 3, 30), cmap=cm.viridis, alpha=0.5)
plt.plot(history[::50, 0], history[::50, 1], 'r.-', markersize=5, linewidth=1)  # Every 50th point
plt.scatter([start[0]], [start[1]], color='green', s=100, marker='o', label='Friday Baseline', zorder=5)
plt.scatter([final[0]], [final[1]], color='red', s=100, marker='*', label='Final Setup', zorder=5)
plt.xlabel('Wing Angle')
plt.ylabel('Ride Height')
plt.title('Setup Optimization on Rosenbrock (Monaco)')
plt.legend()

plt.subplot(1, 2, 2)
losses = [rosenbrock(p) for p in history[::10]]
plt.plot(losses)
plt.xlabel('Iteration (x10)')
plt.ylabel('Lap Time')
plt.title('Lap Time Over Optimization Steps')
plt.yscale('log')

plt.tight_layout()
plt.show()

print("\nNote: Simple gradient descent struggles with narrow valleys!")
print("F1: This is why experienced engineers and advanced tools are needed")
print("for circuits like Monaco with tight setup windows.")
print("More advanced optimizers (Adam, etc.) handle this better.")

---

## 6. Putting It Together: Training a Linear Model

Let's train a simple linear regression model using gradient descent.

**F1 scenario:** We're building a simple model to predict lap time from fuel load. The relationship is roughly linear: heavier fuel means slower laps. We'll use gradient descent to find the best-fit line — exactly how a team might calibrate their performance model from practice data.

In [ ]:
# Generate synthetic F1 data: lap time vs fuel load
np.random.seed(42)
n_laps = 100

# True parameters: each kg of fuel adds ~0.035s per lap, baseline time is 1:30.000
# We'll use scaled values for numerical convenience
fuel_effect_true = 2.5    # scaled fuel-to-laptime sensitivity
baseline_time_true = 1.0  # scaled baseline lap time

# Generate data: lap_time = fuel_effect * fuel_load + baseline + noise
fuel_load = np.random.uniform(-3, 3, n_laps)  # centered fuel load values
lap_time = fuel_effect_true * fuel_load + baseline_time_true + np.random.normal(0, 0.5, n_laps)

plt.figure(figsize=(10, 6))
plt.scatter(fuel_load, lap_time, alpha=0.6, label='Practice lap data')
plt.plot(fuel_load, fuel_effect_true * fuel_load + baseline_time_true, 'r-', linewidth=2, 
         label=f'True model: time = {fuel_effect_true}*fuel + {baseline_time_true}')
plt.xlabel('Fuel Load (centered)')
plt.ylabel('Lap Time (scaled)')
plt.title('F1 Practice Data: Lap Time vs Fuel Load\n(Each dot = one practice lap)')
plt.legend()
plt.show()

In [ ]:
def train_fuel_model(X, y, learning_rate=0.01, n_epochs=100):
    """
    Train a linear fuel-to-laptime model using gradient descent.
    
    Model: predicted_time = w * fuel_load + b
    Loss: MSE = (1/n) * sum((predicted_time - actual_time)^2)
    
    F1 context: The team wants to learn the fuel effect coefficient
    and baseline lap time from practice data.
    """
    n = len(X)
    
    # Initialize parameters (start with no knowledge)
    w = 0.0  # fuel effect coefficient
    b = 0.0  # baseline time
    
    history = {'loss': [], 'w': [], 'b': []}
    
    for epoch in range(n_epochs):
        # Forward pass: predict lap times
        y_pred = w * X + b
        
        # Compute loss (MSE — how wrong are our predictions?)
        loss = np.mean((y_pred - y)**2)
        
        # Compute gradients (which direction improves the model?)
        # d(loss)/dw = (2/n) * sum((y_pred - y) * x)
        # d(loss)/db = (2/n) * sum(y_pred - y)
        dw = (2/n) * np.sum((y_pred - y) * X)
        db = (2/n) * np.sum(y_pred - y)
        
        # Update parameters (gradient descent step)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record history
        history['loss'].append(loss)
        history['w'].append(w)
        history['b'].append(b)
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch:3d}: loss = {loss:.4f}, fuel_effect = {w:.4f}, baseline = {b:.4f}")
    
    return w, b, history

# Train the fuel model
w_learned, b_learned, history = train_fuel_model(fuel_load, lap_time, learning_rate=0.1, n_epochs=100)

print(f"\nLearned: fuel_effect = {w_learned:.4f}, baseline = {b_learned:.4f}")
print(f"True:    fuel_effect = {fuel_effect_true:.4f}, baseline = {baseline_time_true:.4f}")
print(f"\nThe model successfully learned the fuel correction from practice data!")

In [ ]:
# Visualize the training process
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss curve
axes[0].plot(history['loss'])
axes[0].set_xlabel('Training Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Prediction Error Over Training')

# Parameter trajectory
axes[1].plot(history['w'], label='fuel_effect (w)')
axes[1].plot(history['b'], label='baseline (b)')
axes[1].axhline(y=fuel_effect_true, color='blue', linestyle='--', alpha=0.5, 
                label=f'true fuel_effect={fuel_effect_true}')
axes[1].axhline(y=baseline_time_true, color='orange', linestyle='--', alpha=0.5, 
                label=f'true baseline={baseline_time_true}')
axes[1].set_xlabel('Training Epoch')
axes[1].set_ylabel('Parameter Value')
axes[1].set_title('Parameter Convergence\n(Model learns the true values!)')
axes[1].legend()

# Final fit
axes[2].scatter(fuel_load, lap_time, alpha=0.6, label='Practice data')
x_line = np.linspace(-3, 3, 100)
axes[2].plot(x_line, fuel_effect_true * x_line + baseline_time_true, 'g-', linewidth=2, 
             label=f'True: {fuel_effect_true}*fuel + {baseline_time_true}')
axes[2].plot(x_line, w_learned * x_line + b_learned, 'r--', linewidth=2, 
             label=f'Learned: {w_learned:.2f}*fuel + {b_learned:.2f}')
axes[2].set_xlabel('Fuel Load')
axes[2].set_ylabel('Lap Time')
axes[2].set_title('Final Model Fit\n(Red dashed = learned, Green = true)')
axes[2].legend()

plt.tight_layout()
plt.show()

---

## Exercises

### Exercise 1: Implement Gradient Checking for a Tire Degradation Model

**F1 scenario:** You've built an analytical model of tire degradation and computed its gradients by hand. Before trusting those gradients for setup optimization, you need to **verify** them against numerical gradients. This is gradient checking — the engineer's sanity check before making expensive setup changes based on model predictions.

Gradient checking is crucial for debugging backpropagation. Compare analytical gradients with numerical gradients.

In [ ]:
def gradient_check(f, grad_f, point, h=1e-5, threshold=1e-5):
    """
    Compare analytical gradient with numerical gradient.
    Returns True if they match within threshold.
    
    F1 context: Verify that your hand-derived tire model gradients 
    match numerical estimates before using them to make setup decisions.
    """
    point = np.array(point, dtype=float)
    analytical_grad = grad_f(point)
    numerical_grad = compute_gradient(f, point, h)
    
    # Compute relative error
    diff = np.abs(analytical_grad - numerical_grad)
    denom = np.maximum(np.abs(analytical_grad) + np.abs(numerical_grad), 1e-10)
    relative_error = diff / denom
    
    print(f"Setup point: {point}")
    print(f"Analytical gradient:  {analytical_grad}")
    print(f"Numerical gradient:   {numerical_grad}")
    print(f"Relative error: {relative_error}")
    print(f"Max relative error: {np.max(relative_error):.2e}")
    
    return np.all(relative_error < threshold)

# Test on a tire degradation model: f(temp, pressure) = temp³ + 2*temp*pressure + pressure²
# F1: How tire performance depends on temperature and pressure
def tire_degradation(p):
    temp, pressure = p
    return temp**3 + 2*temp*pressure + pressure**2

def tire_degradation_grad(p):
    temp, pressure = p
    return np.array([3*temp**2 + 2*pressure, 2*temp + 2*pressure])

passed = gradient_check(tire_degradation, tire_degradation_grad, [2.0, 3.0])
print(f"\nGradient check passed: {passed}")
print("The analytical gradients are trustworthy for setup optimization!")

### Exercise 2: Implement Softmax and Its Gradient

**F1 scenario:** Your strategy team needs to convert raw "fitness scores" for different tire compounds (soft, medium, hard) into **probabilities** — "What's the probability that each compound is the optimal choice for the next stint?" Softmax is the standard way to do this conversion, and it's critical for classification in ML.

Softmax is critical for classification. Implement it and its gradient.

In [ ]:
def softmax(x):
    """
    Compute softmax: softmax(x)_i = exp(x_i) / sum(exp(x_j))
    Subtract max for numerical stability.
    
    F1 context: Convert tire compound fitness scores into selection probabilities.
    """
    # TODO: Implement softmax
    x_shifted = x - np.max(x)  # For numerical stability
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x)

def softmax_jacobian(x):
    """
    Compute Jacobian of softmax.
    J[i,j] = d(softmax_i)/d(x_j)
    
    Formula: J[i,j] = softmax_i * (delta_ij - softmax_j)
    where delta_ij = 1 if i==j, else 0
    
    F1 context: How does changing the fitness score of one compound 
    affect the selection probability of every compound?
    """
    s = softmax(x)
    n = len(s)
    jacobian = np.zeros((n, n))
    
    # TODO: Implement Jacobian
    for i in range(n):
        for j in range(n):
            if i == j:
                jacobian[i, j] = s[i] * (1 - s[j])
            else:
                jacobian[i, j] = -s[i] * s[j]
    
    return jacobian

# Test — tire compound fitness scores: [soft, medium, hard]
compound_scores = np.array([2.0, 1.0, 0.1])
print(f"Compound fitness scores: {compound_scores}")
print(f"  (Soft=2.0, Medium=1.0, Hard=0.1)")
print(f"\nSelection probabilities: {softmax(compound_scores)}")
print(f"Sum (should be 1): {softmax(compound_scores).sum():.6f}")
print(f"\nJacobian (how each score affects each probability):")
print(f"{softmax_jacobian(compound_scores)}")
print(f"\nF1 interpretation: Soft compound has highest probability ({softmax(compound_scores)[0]:.1%})")
print(f"because it has the highest fitness score.")

### Exercise 3: Gradient Descent with Momentum

**F1 scenario:** Standard gradient descent can get stuck in local minima or oscillate in narrow valleys. Momentum is like giving your setup optimizer "inertia" — it builds up speed in consistent directions and carries through small bumps. In F1 terms, instead of reacting purely to the last practice session's data, momentum lets you carry the "trend" from multiple sessions. If the car has been getting faster with more wing for 3 sessions straight, momentum says "keep going in that direction even if this one session was noisy."

Momentum helps accelerate gradient descent. Implement it!

In [ ]:
def gradient_descent_momentum(f, grad_f, start, learning_rate=0.01, momentum=0.9, n_steps=100):
    """
    Gradient descent with momentum.
    
    v = momentum * v - learning_rate * gradient
    x = x + v
    
    F1 context: The velocity term carries the "trend" from previous sessions,
    helping the optimizer build speed in consistent directions and carry past
    noisy bumps in the setup landscape.
    """
    point = np.array(start, dtype=float)
    velocity = np.zeros_like(point)
    history = [point.copy()]
    
    for i in range(n_steps):
        grad = grad_f(point)
        velocity = momentum * velocity - learning_rate * grad
        point = point + velocity
        history.append(point.copy())
        
    return point, np.array(history)

# Compare regular GD vs GD with momentum on Rosenbrock (the "Monaco" landscape)
start = [-1.0, 1.0]
n_steps = 1000

final_gd, history_gd = gradient_descent_2d(rosenbrock, rosenbrock_grad, start, 
                                            learning_rate=0.001, n_steps=n_steps)
final_mom, history_mom = gradient_descent_momentum(rosenbrock, rosenbrock_grad, start,
                                                    learning_rate=0.001, momentum=0.9, n_steps=n_steps)

print(f"Regular GD final lap time:  {rosenbrock(final_gd):.6f}")
print(f"Momentum GD final lap time: {rosenbrock(final_mom):.6f}")
print(f"\nMomentum finds a better setup in the same number of iterations!")

# Visualize
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.contour(X, Y, Z, levels=np.logspace(0, 3, 30), cmap=cm.viridis, alpha=0.5)
plt.plot(history_gd[::20, 0], history_gd[::20, 1], 'b.-', markersize=3, label='Standard GD')
plt.plot(history_mom[::20, 0], history_mom[::20, 1], 'r.-', markersize=3, label='With Momentum')
plt.xlabel('Wing Angle')
plt.ylabel('Ride Height')
plt.title('Setup Optimization Paths\n(Momentum carries through the narrow valley)')
plt.legend()

plt.subplot(1, 2, 2)
losses_gd = [rosenbrock(p) for p in history_gd]
losses_mom = [rosenbrock(p) for p in history_mom]
plt.plot(losses_gd, 'b-', label='Standard GD')
plt.plot(losses_mom, 'r-', label='With Momentum')
plt.xlabel('Iteration')
plt.ylabel('Lap Time')
plt.title('Lap Time: Standard GD vs Momentum\n(Momentum converges faster)')
plt.yscale('log')
plt.legend()

plt.tight_layout()
plt.show()

---

## Summary

### Key Concepts

| Concept | Mathematical Meaning | F1 Parallel |
|---------|---------------------|-------------|
| **Derivatives** | Measure rate of change — essential for optimization | How lap time changes when you tweak a setup parameter (acceleration is the derivative of speed) |
| **Partial derivatives** | Handle functions of multiple variables | Effect of changing ONE parameter (e.g., wing angle) while holding everything else fixed |
| **The gradient** | Points in the direction of steepest ascent | The engineer's "sensitivity report" — which setup parameter to change to improve lap time fastest |
| **Chain rule** | Compute gradients through composed functions (backprop!) | How engine power flows through torque, tire slip, cornering speed to affect lap time (sequential dependencies) |
| **Gradient descent** | Minimize loss by following the negative gradient | Iteratively tuning car setup across practice sessions to minimize lap time |
| **Learning rate** | How big each optimization step is | How aggressive the engineer's setup changes are (too big = overshoot, too small = slow progress) |
| **Local minima** | A point that's locally optimal but not globally | A "good enough" setup that's not actually the fastest — getting stuck |
| **Momentum** | Build up "velocity" in consistent directions | Carrying the trend from multiple sessions to power through noise and shallow local minima |

### Connection to Deep Learning

- **Forward pass**: Compute function values through the network
- **Loss**: Scalar measuring prediction quality
- **Backward pass**: Apply chain rule to compute gradients
- **Update**: Move parameters in negative gradient direction

### Checklist
- [ ] I can compute derivatives of common functions
- [ ] I understand partial derivatives and gradients
- [ ] I can apply the chain rule to composite functions
- [ ] I can implement gradient descent from scratch
- [ ] I understand the effect of learning rate

---

## Next Steps

Continue to **Part 1.3: Probability & Statistics** where we'll cover:
- Probability distributions
- Bayes' theorem
- Maximum likelihood estimation
- Information theory (entropy, KL divergence)

**F1 preview:** Probability and statistics are how F1 teams make decisions under uncertainty — "What's the probability it will rain in 20 minutes?", "Given the current tire wear rate, what's the optimal pit stop window?", "How confident are we that this setup change actually improved the car vs. random variation?" These are all probability and statistics questions.